In [53]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optmizer

import matplotlib.pyplot as plt

import pandas as pd

tf.get_logger().setLevel('ERROR')

In [14]:
from sklearn.model_selection import train_test_split

In [20]:
RANDOM_STATE = 44
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 44

In [5]:
tweets_df = pd.read_csv('data/train.csv', index_col='id')

In [6]:
tweets_df.head()

,keyword,location,text,target
id,,,,
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [9]:
tweet_data = tweets_df.pop('text')
target = tweets_df.pop('target')

In [16]:
X_train, X_val, y_train, y_val = train_test_split(tweet_data, target, test_size=0.2, random_state=RANDOM_STATE)

In [21]:
raw_dataset_train = tf.data.Dataset.from_tensor_slices((X_train, y_train))

In [24]:
raw_dataset_val = tf.data.Dataset.from_tensor_slices((X_val, y_val))

In [22]:
for feat, targ in raw_dataset_train.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: b'Japan Heat Wave Intensifies \xc2\x89\xc3\x9b\xc3\x92 Death Toll Surges To 55  http://t.co/irpSSresRq', Target: 1
Features: b'@Bang_Me_Up_Guk he was ;-; like he was singing so loud ;-;', Target: 0
Features: b'@MarianKeyes Rubber Mudslide! Still laughing!', Target: 0
Features: b'Armageddon https://t.co/uCSUDk3q1d', Target: 1
Features: b'Beyond all bounds; till inundation rise', Target: 0


In [25]:
len(raw_dataset_train), len(raw_dataset_val)

(6090, 1523)

In [36]:
dataset_train = raw_dataset_train.shuffle(len(raw_dataset_train)).batch(batch_size=batch_size).cache().prefetch(buffer_size=AUTOTUNE)


In [37]:
dataset_val = raw_dataset_val.shuffle(len(raw_dataset_val)).batch(batch_size=batch_size).cache().prefetch(buffer_size=AUTOTUNE)

In [41]:
type(raw_dataset_train)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [44]:
for text_batch, label_batch in dataset_train.take(1):
    for i in range(3):
        print(f'Review: {text_batch.numpy()[i]}')
        label = label_batch.numpy()[i]
        print(f'Label : {label} ({label == 1})')

Review: b'How is it one careless match can start a forest fire but it takes a whole box to start a campfire'
Label : 0 (False)
Review: b'@hrips_k when u do a fatality and like the corpse is still jittering'
Label : 1 (True)
Review: b"@Joelsherman1 DW was on his way to a better career than Chipper and that wasn't derailed until 2014. Shame."
Label : 0 (False)


In [47]:
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [50]:
bert_preprocess_model

In [54]:
text_test = ['OMG Horrible Accident Man Died in Wings of Airplane. http://t.co/xDxDPrcPnS. I am so scared.', 'this is not good']
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

InvalidArgumentError:  Trying to access resource using the wrong type. Expected class tensorflow::lookup::LookupInterface got class tensorflow::lookup::LookupInterface
	 [[{{node StatefulPartitionedCall/StatefulPartitionedCall/bert_tokenizer/StatefulPartitionedCall/WordpieceTokenizeWithOffsets/WordpieceTokenizeWithOffsets/WordpieceTokenizeWithOffsets}}]] [Op:__inference_restored_function_body_14616]

Function call stack:
restored_function_body
